In [ ]:
# 필요 라이브러리 블러오기

import cv2
import numpy as np
import time
import datetime
import sys
import imutils
from imutils.perspective import four_point_transform

In [ ]:
# 신호 찾기

def findTrafficSign(grabbed, frame):

    lower_hsv = np.array([85,100,50])
    upper_hsv = np.array([115,255,255])
    
    if not grabbed:
        print("No input image")
        return
    
    frameArea = frame.shape[0] * frame.shape[1]
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_hsv, upper_hsv)
    #cv2.imshow("in range", mask)
    
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    #cv2.imshow("morphology", mask)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    #cv2.imshow("morphology2", mask)

    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    
    largestArea = 0
    largestRect = None
    
    if len(cnts) > 0:
        
        for cnt in cnts:
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            
            sideOne = np.linalg.norm(box[0]-box[1])
            sideTwo = np.linalg.norm(box[0]-box[3])
            area = sideOne*sideTwo
            
            if area > largestArea:
                largestArea = area
                largestRect = box
        
    if largestArea > frameArea * 0.02:
        cv2.drawContours(frame, [largestRect], 0, (0,0,255), 2)

    cropped = None
    if largestRect is not None:
        cropped = four_point_transform(frame, [largestRect][0])

    return frame, cropped, largestRect

In [ ]:
def Main():      
    
    cmd = '' 
    
    print('Press s to save')
    print('Press q to quit')
    
    video_capture = cv2.VideoCapture(video)
    
    while(True):

        grabbed, frame = video_capture.read()
        frame, cropped, largestRect = findTrafficSign(grabbed, frame)
        cv2.imshow('IP Webcam', frame)
        
        if cropped is not None:
            cv2.imshow("cropped", cropped)
                   
        key = cv2.waitKey(1);
        if key == ord('q'):
            break
                
        elif key == ord('s'):
            if cropped is not None:
                file = datetime.datetime.now().strftime("%Y%m%d_%H%M%S%f") +'.jpg'
                cv2.imwrite(file, cropped)
                print(file, ' saved')

    video_capture.release()
    cv2.destroyAllWindows()


In [ ]:
video = 0 #0 = 웹캠

if __name__ == '__main__':
    Main()